In [ ]:
!pip install imutils 
!pip install gdown

In [ ]:
!rm -r MIRNet-TF2

In [ ]:
!git clone https://github.com/shilu10/MIRNet-TF2.git

In [ ]:
!mv MIRNet-TF2 MIRNet_W  

In [2]:
!python train_enhancement.py --loss_function=l1 --n_epochs=150 --batch_size=5 

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [ ]:
!python train_denoise.py --num_rrg=3 --num_mrb=2 --n_epochs=150 --batch_size=5 

In [ ]:
!python train_super_resolution.py --batch_size=10 --n_epochs=100 

In [ ]:
!pip install tqdm

In [ ]:
import tqdm 
import glob
import matplotlib.pyplot as plt
import cv2

In [ ]:
!cp MIRNet-TF2/test_enhancement.py MIRNet-TF2/test_denoise.py MIRNet-TF2/test_super_resolution.py MIRNet-TF2/utils.py  MIRNet-TF2/tflite_inference.py .

In [ ]:
# inference for all tf- models:

!python test_super_resolution.py --test_path="test/super_resolution/" --file_extension="png" --plot_results=True

In [ ]:
for img_path in glob.glob("results/super_resolution/*.png"):
    plt.figure()
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

In [ ]:
sr_data_path = '.div2k/images/DIV2K_valid_LR_bicubic/X4/'
sr_result_path = "results/new/"

In [ ]:
glob.glob(sr_data_path+"*.png")

In [ ]:
!python test_super_resolution.py 

In [ ]:
import tensorflow as tf

In [ ]:
count = 0
for img_path in glob.glob("results/super_resolution/*.png"):
    img_fname = img_path.split("/")[-1]
    
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    hr_img = cv2.imread(f".div2k/images/DIV2K_valid_HR/{img_fname.replace('x4', '')}")
    hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2RGB)
    hr_img = cv2.resize(hr_img, (400*4, 260*4))
    
    print("PSNR: ", tf.image.psnr(img, hr_img, max_val=255))
    
    if count == 10:
        break
    plt.figure(figsize=(15, 15))
    plt.subplot(121)
    plt.axis("off")
    plt.imshow(img)
    
    plt.subplot(122)
    plt.axis("off")
    plt.imshow(hr_img)
    plt.show()
    count += 1

In [ ]:
!git clone https://github.com/tuvovan/MIRNet-Keras.git

In [ ]:
!python test_denoise.py --file_extension="PNG"

In [ ]:
for img_path in glob.glob("results/denoise/*.PNG"):
    img_fname = img_path.split('/')[-1].replace('NOISY', "GT")
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    hr_img = cv2.imread(f"gt/denoise/{img_fname}")
    hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2RGB)
    hr_img = cv2.resize(hr_img, (512, 360))
    
    print(img.shape, hr_img.shape)
    print("PSNR: ", tf.image.psnr(img, hr_img, max_val=255))
    
    plt.figure(figsize=(15, 15))
    plt.title('Prediction')
    plt.subplot(121)
    plt.axis('off')
    plt.imshow(img)
    
    plt.title('Ground Truth')
    plt.subplot(122)
    plt.axis('off')
    plt.imshow(hr_img)
    plt.show()

In [ ]:
# LIME data

!python test_enhancement.py 

In [ ]:
import glob
import cv2 
import matplotlib.pyplot as plt 

for img_path in glob.glob("results/LIME/*.bmp"):
    plt.figure()
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

In [ ]:
!python test_enhancement.py --test_path="test/enhancement/" --save_path="results/enhancement/" --file_extension="png"

In [ ]:
for img_path in glob.glob("results/enhancement/*.png"):
    plt.figure()
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

In [ ]:
# tflite conversion.

In [ ]:
!python tflite_conversion.py --saved_model_path="checkpoint/denoise/best_model.h5" \
                            --tflite_model_path="checkpoint/denoise/best_model.tflite" \
                            --num_mrb=2 \
                            --num_rrg=3

In [ ]:
!python tflite_conversion.py --saved_model_path="checkpoint/enhancement/best_model.h5" \
                            --tflite_model_path="/kaggle/tflite_models/best_model.tflite" \
                            --num_mrb=2 \
                            --num_rrg=3

In [ ]:
!python tflite_conversion.py --saved_model_path="checkpoint/super_resolution/best_model.h5" \
                            --tflite_model_path="/kaggle/tflite_models/best_model_sr.tflite" \
                            --num_mrb=2 \
                            --num_rrg=3

In [ ]:
# tflite inference.

In [ ]:
!python tflite_inference.py --tflite_model_path="checkpoint/denoise/best_model.tflite" \
                            --test_data_path="test/denoise/" \
                            --save_path="results/denoise/" \
                            --mode="denoise" \
                            --file_extension="PNG"

In [ ]:
for img_path in glob.glob("results/denoise/*.PNG"):
    plt.figure()
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

In [ ]:
!python tflite_inference.py --tflite_model_path="/kaggle/tflite_models/best_model.tflite" \
                            --test_data_path="test/enhancement/" \ 
                            --save_path="results/tflite/enhancement/" \
                            --mode="enhancement" \
                            --file_extension="png"

In [ ]:
!python tflite_inference.py --tflite_model_path="/kaggle/tflite_models/best_model_sr.tflite" \
                            --test_data_path="test/super_resolution/" \ 
                            --save_path="results/tflite/super_resolution/" \
                            --mode="super_resolution" \
                            --file_extension="png"

In [ ]:
!rm -r /kaggle/working/MIRNet-TF2/

In [ ]:
!rm -r /kaggle/working/MIRNet_W/SIDD_Small_sRGB_Only